In [149]:
import gym
import os,sys,inspect
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from datetime import datetime
from agents.Base import deepagents, run
from agents.BlueAgents import *
from agents.agents.rec_ppo import AgentSharedRecurrentPPO
from agents.Base.run import Arguments, train_and_evaluate

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
env = gym.make("CartPole-v0")

In [151]:
recurrent_config = {'recurrent_n_layer':1, 'use_orthogonal':True,
                    'sequence_length':8, 'bidirectional':False, 'hidden_state_size':64}
model = AgentSharedRecurrentPPO(recurrent_config)
model_kwargs = {}   


            
"""model.learning_rate = model_kwargs["learning_rate"]
model.batch_size = model_kwargs["batch_size"]
model.gamma = model_kwargs["gamma"]
model.seed = model_kwargs["seed"]
model.net_dim = model_kwargs["net_dimension"]
model.target_step = model_kwargs["target_step"]
model.eval_gap = model_kwargs["eval_time_gap"]
model.episode = model_kwargs["episode"]"""

'model.learning_rate = model_kwargs["learning_rate"]\nmodel.batch_size = model_kwargs["batch_size"]\nmodel.gamma = model_kwargs["gamma"]\nmodel.seed = model_kwargs["seed"]\nmodel.net_dim = model_kwargs["net_dimension"]\nmodel.target_step = model_kwargs["target_step"]\nmodel.eval_gap = model_kwargs["eval_time_gap"]\nmodel.episode = model_kwargs["episode"]'

In [152]:
args.agent.ClassCri

In [153]:
env.action_space

Discrete(2)

In [154]:
def write_summary(logger, episode) -> None:
    """Writes to an event file based on the run-id argument.
            Args:
                update {int} -- Current PPO Update
                training_stats {list} -- Statistics of the training algorithm
                episode_result {dict} -- Statistics of completed episodes
    """
    writer = SummaryWriter()

    for key, value in logger.items():
        
        print(f'{key} is {value} for episode {episode}')
        #writer.add_scalar(f"episode_losses/{key}", value, episode)
        
def get_step_r_exp(ten_reward):
    return len(ten_reward), ten_reward.mean().item()

def train(config):
    env_name = config['env']
    model = config['model']
    
    
    if env_name == 'cartpole':
        env = gym.make('CartPole-v0')
    elif env_name == 'pendulum':
        env = gym.make('Pendulum-v0')
    else:
        raise NotImplementedError
        
    target_step = config['target_step']
    reward_scale = config['reward_scale']
    gamma = config['gamma']
    batch_size = config['batch_size']
    repeat_times = config['repeat_times']
    soft_update_tau = config['soft_update_tau']
    episodes = config['episodes']
    env.max_step = target_step
    env.state_dim = env.observation_space.shape[0]
    env.if_discrete = True

    env.action_dim = env.action_space.shape[0]
    
    args = Arguments(env, env, model)
    
    args.cwd = '/evaluate_envs'
    args.learner_id = 1
    args.init_before_training()
        
    buffer = list()
    
    def update_buffer(traj):
        pass
    model = args.agent
    learning_rate_start = 0.004
    model.init( net_dim=args.net_dim, gpu_id='1',
               state_dim=args.state_dim, action_dim=args.action_dim, env_num=args.env_num,
               learning_rate=learning_rate_start, if_per_or_gae=args.if_per_or_gae)
    
    
    model.states = [np.array(env.reset()), ]
    
    
    buffer = list()
    def update_buffer(_traj_list):
        (ten_state, ten_hidden_state_in,
          ten_hidden_state_out,
          ten_reward, ten_mask, ten_action, ten_noise, ten_values) = _traj_list[0]

        buffer [:] = (ten_state.squeeze(1),
                      ten_hidden_state_in, ten_hidden_state_out,
                      ten_reward, ten_mask, ten_action.squeeze(1),
                      ten_noise.squeeze(1), ten_values.squeeze(1))

        _steps, _r_exp = get_step_r_exp(ten_reward=ten_state[0])  # other = (reward, mask, action)
        return _steps, _r_exp
    for episode in range(episodes):
        with torch.no_grad():

            traje = model.explore_one_env(env, target_step, reward_scale, gamma)

            steps, r_exp = update_buffer(traje)

        logging_dict = model.update_net(buffer, batch_size, repeat_times, soft_update_tau)
        
        #Log 
        write_summary(logging_dict, episode)
        print(f'Episode step: {steps} and reward: {r_exp}')
        print('='*60)
    
    
        

In [155]:
config = {
    'env':'pendulum',
    'gamma':1,
    'batch_size':16,
    'soft_update_tau':0.1,
    'episodes':5,
    'repeat_times':5,
    'reward_scale':1,
    'target_step':1000,
    'model':model
    
}

In [156]:
train(config)

| Remove cwd: /evaluate_envs
Using device:  cpu
tensor([-0.0394])
tensor([-0.0415])
tensor([-0.0406])
tensor([-0.0385])
tensor([-0.0356])
tensor([-0.0320])
tensor([-0.0279])
tensor([-0.0236])
tensor([-0.0192])
tensor([-0.0151])
tensor([-0.0113])
tensor([-0.0080])
tensor([-0.0052])
tensor([-0.0028])
tensor([-0.0010])
tensor([0.0003])
tensor([0.0010])
tensor([0.0014])
tensor([0.0013])
tensor([0.0008])
tensor([0.0001])
tensor([-0.0008])
tensor([-0.0019])
tensor([-0.0032])
tensor([-0.0049])
tensor([-0.0070])
tensor([-0.0095])
tensor([-0.0123])
tensor([-0.0156])
tensor([-0.0192])
tensor([-0.0231])
tensor([-0.0272])
tensor([-0.0316])
tensor([-0.0360])
tensor([-0.0403])
tensor([-0.0443])
tensor([-0.0478])
tensor([-0.0505])
tensor([-0.0523])
tensor([-0.0531])
tensor([-0.0530])
tensor([-0.0518])
tensor([-0.0498])
tensor([-0.0470])
tensor([-0.0433])
tensor([-0.0391])
tensor([-0.0344])
tensor([-0.0296])
tensor([-0.0248])
tensor([-0.0201])
tensor([-0.0157])
tensor([-0.0116])
tensor([-0.0077])
tens

tensor([0.0161])
tensor([0.0175])
tensor([0.0190])
tensor([0.0205])
tensor([0.0225])
tensor([0.0251])
tensor([0.0288])
tensor([0.0342])
tensor([0.0421])
tensor([0.0515])
tensor([0.0542])
tensor([0.0442])
tensor([0.0324])
tensor([0.0240])
tensor([0.0184])
tensor([0.0146])
tensor([0.0118])
tensor([0.0096])
tensor([0.0077])
tensor([0.0062])
tensor([0.0047])
tensor([0.0033])
tensor([0.0019])
tensor([0.0005])
tensor([-0.0010])
tensor([-0.0023])
tensor([-0.0030])
tensor([-0.0019])
tensor([0.0015])
tensor([0.0051])
tensor([0.0080])
tensor([0.0104])
tensor([0.0124])
tensor([0.0141])
tensor([0.0157])
tensor([0.0171])
tensor([0.0153])
tensor([0.0140])
tensor([0.0124])
tensor([0.0096])
tensor([0.0080])
tensor([0.0080])
tensor([0.0091])
tensor([0.0106])
tensor([0.0121])
tensor([0.0135])
tensor([0.0146])
tensor([0.0154])
tensor([0.0161])
tensor([0.0168])
tensor([0.0177])
tensor([0.0191])
tensor([0.0214])
tensor([0.0250])
tensor([0.0310])
tensor([0.0402])
tensor([0.0469])
tensor([0.0379])
tensor([0.

tensor([-0.7612], grad_fn=<SelectBackward0>)
tensor([-0.5589], grad_fn=<SelectBackward0>)
tensor([-0.1755], grad_fn=<SelectBackward0>)
tensor([0.4268], grad_fn=<SelectBackward0>)
tensor([0.1207], grad_fn=<SelectBackward0>)
tensor([0.1734], grad_fn=<SelectBackward0>)
tensor([0.2619], grad_fn=<SelectBackward0>)
tensor([0.3368], grad_fn=<SelectBackward0>)
tensor([0.1204], grad_fn=<SelectBackward0>)
tensor([-0.6156], grad_fn=<SelectBackward0>)
tensor([0.3059], grad_fn=<SelectBackward0>)
tensor([0.1595], grad_fn=<SelectBackward0>)
tensor([-0.3361], grad_fn=<SelectBackward0>)
tensor([0.1913], grad_fn=<SelectBackward0>)
tensor([-0.0154], grad_fn=<SelectBackward0>)
tensor([0.5234], grad_fn=<SelectBackward0>)
tensor([0.4529], grad_fn=<SelectBackward0>)
tensor([0.4246], grad_fn=<SelectBackward0>)
tensor([-0.0755], grad_fn=<SelectBackward0>)
tensor([0.2813], grad_fn=<SelectBackward0>)
tensor([-0.1649], grad_fn=<SelectBackward0>)
tensor([-0.2631], grad_fn=<SelectBackward0>)
tensor([0.4925], grad_f

tensor([-0.2232])
tensor([-0.2230])
tensor([-0.2223])
tensor([-0.2213])
tensor([-0.2202])
tensor([-0.2191])
tensor([-0.2181])
tensor([-0.2172])
tensor([-0.2163])
tensor([-0.2155])
tensor([-0.2146])
tensor([-0.2139])
tensor([-0.2133])
tensor([-0.2130])
tensor([-0.2127])
tensor([-0.2125])
tensor([-0.2124])
tensor([-0.2126])
tensor([-0.2134])
tensor([-0.2149])
tensor([-0.2170])
tensor([-0.2191])
tensor([-0.2210])
tensor([-0.2225])
tensor([-0.2232])
tensor([-0.2231])
tensor([-0.2225])
tensor([-0.2215])
tensor([-0.2204])
tensor([-0.2192])
tensor([-0.2183])
tensor([-0.2174])
tensor([-0.2165])
tensor([-0.2156])
tensor([-0.2147])
tensor([-0.2140])
tensor([-0.2134])
tensor([-0.2130])
tensor([-0.2127])
tensor([-0.2125])
tensor([-0.2124])
tensor([-0.2126])
tensor([-0.2133])
tensor([-0.2148])
tensor([-0.2168])
tensor([-0.2189])
tensor([-0.2209])
tensor([-0.2224])
tensor([-0.2231])
tensor([-0.2231])
tensor([-0.2226])
tensor([-0.2216])
tensor([-0.2205])
tensor([-0.2193])
tensor([-0.2183])
tensor([-0

tensor([-0.2160])
tensor([-0.2168])
tensor([-0.2172])
tensor([-0.2175])
tensor([-0.2175])
tensor([-0.2173])
tensor([-0.2166])
tensor([-0.2153])
tensor([-0.2133])
tensor([-0.2103])
tensor([-0.2055])
tensor([-0.1989])
tensor([-0.1910])
tensor([-0.1823])
tensor([-0.1731])
tensor([-0.1642])
tensor([-0.1566])
tensor([-0.1512])
tensor([-0.1488])
tensor([-0.1495])
tensor([-0.1525])
tensor([-0.1571])
tensor([-0.1624])
tensor([-0.1682])
tensor([-0.1742])
tensor([-0.1801])
tensor([-0.1859])
tensor([-0.1913])
tensor([-0.1963])
tensor([-0.2008])
tensor([-0.2045])
tensor([-0.2074])
tensor([-0.2099])
tensor([-0.2120])
tensor([-0.2138])
tensor([-0.2151])
tensor([-0.2161])
tensor([-0.2169])
tensor([-0.2173])
tensor([-0.2176])
tensor([-0.2176])
tensor([-0.2173])
tensor([-0.2166])
tensor([-0.2153])
tensor([-0.2132])
tensor([-0.2102])
tensor([-0.2053])
tensor([-0.1986])
tensor([-0.1906])
tensor([-0.1817])
tensor([-0.1724])
tensor([-0.1632])
tensor([-0.1554])
tensor([-0.1498])
tensor([-0.1475])
tensor([-0

...

tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2341])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2339])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1.2340])
tensor([-1

tensor([-1.2314])
tensor([-1.2314])
tensor([-1.2314])
tensor([-1.2313])
tensor([-1.2313])
tensor([-1.2312])
tensor([-1.2312])
tensor([-1.2312])
tensor([-1.2311])
tensor([-1.2311])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2309])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2310])
tensor([-1.2311])
tensor([-1.2311])
tensor([-1.2311])
tensor([-1.2311])
tensor([-1.2312])
tensor([-1.2312])
tensor([-1.2312])
tensor([-1.2313])
tensor([-1.2314])
tensor([-1.2314])
tensor([-1.2315])
tensor([-1.2315])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2316])
tensor([-1.2315])
tensor([-1.2315])
tensor([-1

tensor([-11.2244], grad_fn=<SelectBackward0>)
tensor([-11.2589], grad_fn=<SelectBackward0>)
tensor([-11.2925], grad_fn=<SelectBackward0>)
tensor([-11.3036], grad_fn=<SelectBackward0>)
tensor([-11.3424], grad_fn=<SelectBackward0>)
tensor([-11.3680], grad_fn=<SelectBackward0>)
tensor([-11.3939], grad_fn=<SelectBackward0>)
tensor([-11.4055], grad_fn=<SelectBackward0>)
tensor([-11.4240], grad_fn=<SelectBackward0>)
tensor([-11.4376], grad_fn=<SelectBackward0>)
tensor([-11.4533], grad_fn=<SelectBackward0>)
tensor([-11.4645], grad_fn=<SelectBackward0>)
tensor([-11.4650], grad_fn=<SelectBackward0>)
tensor([-11.4862], grad_fn=<SelectBackward0>)
tensor([-11.4919], grad_fn=<SelectBackward0>)
tensor([-11.5032], grad_fn=<SelectBackward0>)
tensor([-11.5134], grad_fn=<SelectBackward0>)
tensor([-11.4994], grad_fn=<SelectBackward0>)
tensor([-11.5263], grad_fn=<SelectBackward0>)
tensor([-11.5290], grad_fn=<SelectBackward0>)
tensor([-11.5338], grad_fn=<SelectBackward0>)
tensor([-11.5411], grad_fn=<Select

tensor([-11.6112])
tensor([-11.6112])
tensor([-11.6112])
tensor([-11.6113])
tensor([-11.6115])
tensor([-11.6118])
tensor([-11.6122])
tensor([-11.6125])
tensor([-11.6127])
tensor([-11.6129])
tensor([-11.6130])
tensor([-11.6130])
tensor([-11.6129])
tensor([-11.6126])
tensor([-11.6123])
tensor([-11.6119])
tensor([-11.6115])
tensor([-11.6114])
tensor([-11.6116])
tensor([-11.6116])
tensor([-11.6116])
tensor([-11.6115])
tensor([-11.6116])
tensor([-11.6118])
tensor([-11.6121])
tensor([-11.6125])
tensor([-11.6128])
tensor([-11.6131])
tensor([-11.6133])
tensor([-11.6134])
tensor([-11.6133])
tensor([-11.6132])
tensor([-11.6130])
tensor([-11.6127])
tensor([-11.6123])
tensor([-11.6119])
tensor([-11.6179])
tensor([-11.6185])
tensor([-11.6189])
tensor([-11.6194])
tensor([-11.6198])
tensor([-11.6202])
tensor([-11.6205])
tensor([-11.6207])
tensor([-11.6208])
tensor([-11.6209])
tensor([-11.6208])
tensor([-11.6206])
tensor([-11.6203])
tensor([-11.6200])
tensor([-11.6196])
tensor([-11.6192])
tensor([-11.

tensor([-11.6204])
tensor([-11.6207])
tensor([-11.6209])
tensor([-11.6212])
tensor([-11.6214])
tensor([-11.6216])
tensor([-11.6218])
tensor([-11.6219])
tensor([-11.6221])
tensor([-11.6222])
tensor([-11.6224])
tensor([-11.6225])
tensor([-11.6227])
tensor([-11.6229])
tensor([-11.6231])
tensor([-11.6233])
tensor([-11.6236])
tensor([-11.6239])
tensor([-11.6243])
tensor([-11.6247])
tensor([-11.6252])
tensor([-11.6257])
tensor([-11.6263])
tensor([-11.6269])
tensor([-11.6274])
tensor([-11.6278])
tensor([-11.6281])
tensor([-11.6281])
tensor([-11.6280])
tensor([-11.6277])
tensor([-11.6272])
tensor([-11.6266])
tensor([-11.6260])
tensor([-11.6252])
tensor([-11.6245])
tensor([-11.6238])
tensor([-11.6230])
tensor([-11.6223])
tensor([-11.6217])
tensor([-11.6210])
tensor([-11.6204])
tensor([-11.6198])
tensor([-11.6193])
tensor([-11.6189])
tensor([-11.6185])
tensor([-11.6183])
tensor([-11.6182])
tensor([-11.6183])
tensor([-11.6184])
tensor([-11.6187])
tensor([-11.6191])
tensor([-11.6194])
tensor([-11.

In [89]:
!pip install atari-py